Load in data from turnstile file

In [54]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns

%matplotlib inline

# set display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

In [81]:
# Source: http://web.mta.info/developers/turnstile.html
def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs)
        
#week_nums = [180224,180303, 180310, 180317, 180324, 180331, 180407, 180414, 180421, 180428, 180505, 180512, 180519, 180526, 180602]
week_nums =[180303, 180310, 180317, 180324, 180331]

turnstiles_df = get_data(week_nums)

In [82]:
#check the top enteries to see the data
turnstiles_df.head()

C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  03:00:00   
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  07:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  11:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  15:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  19:00:00   

         DESC  ENTRIES  \
0  RECOVR AUD  6527780   
1     REGULAR  6527794   
2     REGULAR  6527838   
3     REGULAR  6527994   
4     REGULAR  6528278   

   EXITS                                                                 
0                                            2210496                     
1                                            2210507                     
2                                            2210585                     
3                                            2210644                     
4                                            2210705

In [57]:
#check the dimension of the data
turnstiles_df.shape

(987294, 11)

In [83]:
turnstiles_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')

In [84]:
#list comprehension that strips the extra space from the column names
turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]

In [85]:
turnstiles_df.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [86]:
# Take the date and time fields into a single datetime column
turnstiles_df["DATE_TIME"] = pd.to_datetime(turnstiles_df.DATE + " " + turnstiles_df.TIME, format="%m/%d/%Y %H:%M:%S")

In [70]:
#check entries for duplicates
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])  
 .ENTRIES.count()
 .reset_index() # or use as_index = False; otherwise makes groupby columns new index 
 .sort_values("ENTRIES", ascending=False)).head(5)

C/A  UNIT       SCP         STATION           DATE_TIME  ENTRIES
648338  R135  R031  01-00-00  34 ST-PENN STA 2018-03-22 08:00:00        2
0       A002  R051  02-00-00           59 ST 2018-02-24 03:00:00        1
658201  R143  R032  02-03-01  TIMES SQ-42 ST 2018-03-17 16:00:00        1
658189  R143  R032  02-03-01  TIMES SQ-42 ST 2018-03-15 16:00:00        1
658190  R143  R032  02-03-01  TIMES SQ-42 ST 2018-03-15 20:00:00        1

In [71]:
mask = ((turnstiles_df["C/A"] == "R135") & 
        (turnstiles_df["UNIT"] == "R031") & 
        (turnstiles_df["SCP"] == "01-00-00") & 
        (turnstiles_df["STATION"] == "34 ST-PENN STA") &
        (turnstiles_df["DATE_TIME"] == "2018-03-22 08:00:00"))

turnstiles_df[mask].head()


C/A  UNIT       SCP         STATION LINENAME DIVISION        DATE  \
129506  R135  R031  01-00-00  34 ST-PENN STA      123      IRT  03/22/2018   
129507  R135  R031  01-00-00  34 ST-PENN STA      123      IRT  03/22/2018   

            TIME        DESC   ENTRIES     EXITS           DATE_TIME  
129506  08:00:00     REGULAR  14789593  20329631 2018-03-22 08:00:00  
129507  08:00:00  RECOVR AUD  14789592  20329631 2018-03-22 08:00:00

In [73]:
#check exits for duplicates
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])  
 .EXITS.count()
 .reset_index() # or use as_index = False; otherwise makes groupby columns new index 
 .sort_values("EXITS", ascending=False)).head(5)

###It does not matter which one you take thay all return the ame result.

C/A  UNIT       SCP         STATION           DATE_TIME  EXITS
648338  R135  R031  01-00-00  34 ST-PENN STA 2018-03-22 08:00:00      2
0       A002  R051  02-00-00           59 ST 2018-02-24 03:00:00      1
658201  R143  R032  02-03-01  TIMES SQ-42 ST 2018-03-17 16:00:00      1
658189  R143  R032  02-03-01  TIMES SQ-42 ST 2018-03-15 16:00:00      1
658190  R143  R032  02-03-01  TIMES SQ-42 ST 2018-03-15 20:00:00      1

In [88]:
turnstiles_df['DESC'].unique()
turnstiles_df = turnstiles_df[turnstiles_df['DESC'] == 'REGULAR']
#turnstiles_df.shape

In [90]:
(turnstiles_df
 .groupby(["C/A", "UNIT", "SCP", "STATION", "DATE_TIME"])  
 .EXITS.count()
 .reset_index() # or use as_index = False; otherwise makes groupby columns new index 
 .sort_values("EXITS", ascending=False)).head(5)

C/A  UNIT       SCP         STATION           DATE_TIME  EXITS
0       A002  R051  02-00-00           59 ST 2018-02-24 07:00:00      1
654617  R143  R032  02-03-01  TIMES SQ-42 ST 2018-02-27 19:00:00      1
654605  R143  R032  02-03-01  TIMES SQ-42 ST 2018-02-25 19:00:00      1
654606  R143  R032  02-03-01  TIMES SQ-42 ST 2018-02-25 23:00:00      1
654607  R143  R032  02-03-01  TIMES SQ-42 ST 2018-02-26 03:00:00      1

In [91]:
turnstiles_df.head()

C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  07:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  11:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  15:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  19:00:00   
5  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  23:00:00   

      DESC  ENTRIES    EXITS           DATE_TIME  
1  REGULAR  6527794  2210507 2018-02-24 07:00:00  
2  REGULAR  6527838  2210585 2018-02-24 11:00:00  
3  REGULAR  6527994  2210644 2018-02-24 15:00:00  
4  REGULAR  6528278  2210705 2018-02-24 19:00:00  
5  REGULAR  6528389  2210728 2018-02-24 23:00:00

In [92]:
turnstiles_df = turnstiles_df.drop(["DESC"], axis=1, errors="ignore")

In [93]:
turnstiles_df.head()

C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  07:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  11:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  15:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  19:00:00   
5  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  23:00:00   

   ENTRIES    EXITS           DATE_TIME  
1  6527794  2210507 2018-02-24 07:00:00  
2  6527838  2210585 2018-02-24 11:00:00  
3  6527994  2210644 2018-02-24 15:00:00  
4  6528278  2210705 2018-02-24 19:00:00  
5  6528389  2210728 2018-02-24 23:00:00

In [96]:
#turnstiles entries daily
turnstiles_daily_entries = turnstiles_df.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"])\
.ENTRIES.first().reset_index()

In [97]:
turnstiles_daily_entries.head()

C/A  UNIT       SCP STATION        DATE  ENTRIES
0  A002  R051  02-00-00   59 ST  02/24/2018  6527794
1  A002  R051  02-00-00   59 ST  02/25/2018  6528417
2  A002  R051  02-00-00   59 ST  02/26/2018  6528905
3  A002  R051  02-00-00   59 ST  02/27/2018  6530309
4  A002  R051  02-00-00   59 ST  02/28/2018  6531774

In [98]:
turnstiles_daily_exits = turnstiles_df.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"])\
.EXITS.first().reset_index()

In [99]:
turnstiles_daily_exits.head()

C/A  UNIT       SCP STATION        DATE    EXITS
0  A002  R051  02-00-00   59 ST  02/24/2018  2210507
1  A002  R051  02-00-00   59 ST  02/25/2018  2210734
2  A002  R051  02-00-00   59 ST  02/26/2018  2210907
3  A002  R051  02-00-00   59 ST  02/27/2018  2211297
4  A002  R051  02-00-00   59 ST  02/28/2018  2211800

In [100]:
turnstiles_daily = turnstiles_df.groupby(["C/A", "UNIT", "SCP", "STATION", "DATE"])\
.ENTRIES.first()
.EXITS.first().reset_index()

SyntaxError: invalid syntax (<ipython-input-100-3b7c82ab780b>, line 2)

In [ ]:
turnstiles_df.DATE.value_counts().sort_index()

In [62]:
#masking
mask = ((turnstiles_df["C/A"] == "A002") & 
        (turnstiles_df["UNIT"] == "R051") & 
        (turnstiles_df["SCP"] == "02-00-00") & 
        (turnstiles_df["STATION"] == "59 ST"))

turnstiles_df[mask].head()

C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  03:00:00   
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  07:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  11:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  15:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  02/24/2018  19:00:00   

         DESC  ENTRIES    EXITS  
0  RECOVR AUD  6527780  2210496  
1     REGULAR  6527794  2210507  
2     REGULAR  6527838  2210585  
3     REGULAR  6527994  2210644  
4     REGULAR  6528278  2210705

In [ ]:
# load data from MTA
#df = pd.read_csv('./data/turnstile_180922.txt')

# convert time data into datetime objects
#df['TIMING'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'],format = '%m/%d/%Y %H:%M:%S' )

In [ ]:
df.head()

In [64]:
(df.groupby(
['C/A', 'UNIT','SCP','STATION', 'TIMING'])
.sum()
.reset_index()
)

C/A  UNIT       SCP        STATION              TIMING  ENTRIES  \
0        A002  R051  02-00-00          59 ST 2018-09-15 00:00:00  6759219   
1        A002  R051  02-00-00          59 ST 2018-09-15 04:00:00  6759234   
2        A002  R051  02-00-00          59 ST 2018-09-15 08:00:00  6759251   
3        A002  R051  02-00-00          59 ST 2018-09-15 12:00:00  6759330   
4        A002  R051  02-00-00          59 ST 2018-09-15 16:00:00  6759538   
5        A002  R051  02-00-00          59 ST 2018-09-15 20:00:00  6759808   
6        A002  R051  02-00-00          59 ST 2018-09-16 00:00:00  6759951   
7        A002  R051  02-00-00          59 ST 2018-09-16 04:00:00  6759968   
8        A002  R051  02-00-00          59 ST 2018-09-16 08:00:00  6759987   
9        A002  R051  02-00-00          59 ST 2018-09-16 12:00:00  6760045   
10       A002  R051  02-00-00          59 ST 2018-09-16 16:00:00  6760160   
11       A002  R051  02-00-00          59 ST 2018-09-16 20:00:00  6760301   
12       A002  R051  02-00-00          59 ST 2018-09-17 00:00:00  6760373   
13       A002  R051  02-00-00          59 ST 2018-09-17 04:00:00  6760378   
14       A002  R051  02-00-00          59 ST 2018-09-17 08:00:00  6760412   
15       A002  R051  02-00-00          59 ST 2018-09-17 12:00:00  6760526   
16       A002  R051  02-00-00          59 ST 2018-09-17 16:00:00  6760809   
17       A002  R051  02-00-00          59 ST 2018-09-17 20:00:00  6761564   
18       A002  R051  02-00-00          59 ST 2018-09-18 00:00:00  6761730   
19       A002  R051  02-00-00          59 ST 2018-09-18 04:00:00  6761745   
20       A002  R051  02-00-00          59 ST 2018-09-18 08:00:00  6761778   
21       A002  R051  02-00-00          59 ST 2018-09-18 12:00:00  6761913   
22       A002  R051  02-00-00          59 ST 2018-09-18 16:00:00  6762203   
23       A002  R051  02-00-00          59 ST 2018-09-18 20:00:00  6762919   
24       A002  R051  02-00-00          59 ST 2018-09-19 00:00:00  6763079   
25       A002  R051  02-00-00          59 ST 2018-09-19 04:00:00  6763097   
26       A002  R051  02-00-00          59 ST 2018-09-19 08:00:00  6763140   
27       A002  R051  02-00-00          59 ST 2018-09-19 12:00:00  6763278   
28       A002  R051  02-00-00          59 ST 2018-09-19 16:00:00  6763565   
29       A002  R051  02-00-00          59 ST 2018-09-19 20:00:00  6764299   
30       A002  R051  02-00-00          59 ST 2018-09-20 00:00:00  6764482   
31       A002  R051  02-00-00          59 ST 2018-09-20 04:00:00  6764494   
32       A002  R051  02-00-00          59 ST 2018-09-20 07:26:01  6764512   
33       A002  R051  02-00-00          59 ST 2018-09-20 08:00:00  6764529   
34       A002  R051  02-00-00          59 ST 2018-09-20 12:00:00  6764691   
35       A002  R051  02-00-00          59 ST 2018-09-20 16:00:00  6764991   
36       A002  R051  02-00-00          59 ST 2018-09-20 20:00:00  6765784   
37       A002  R051  02-00-00          59 ST 2018-09-21 00:00:00  6765976   
38       A002  R051  02-00-00          59 ST 2018-09-21 04:00:00  6765985   
39       A002  R051  02-00-00          59 ST 2018-09-21 08:00:00  6766026   
40       A002  R051  02-00-00          59 ST 2018-09-21 12:00:00  6766182   
41       A002  R051  02-00-00          59 ST 2018-09-21 16:00:00  6766532   
42       A002  R051  02-00-00          59 ST 2018-09-21 20:00:00  6767315   
43       A002  R051  02-00-01          59 ST 2018-09-15 00:00:00  6045231   
44       A002  R051  02-00-01          59 ST 2018-09-15 04:00:00  6045246   
45       A002  R051  02-00-01          59 ST 2018-09-15 08:00:00  6045257   
46       A002  R051  02-00-01          59 ST 2018-09-15 12:00:00  6045323   
47       A002  R051  02-00-01          59 ST 2018-09-15 16:00:00  6045486   
48       A002  R051  02-00-01          59 ST 2018-09-15 20:00:00  6045716   
49       A002  R051  02-00-01          59 ST 2018-09-16 00:00:00  6045808   
50       A002  R051  02-00-01          59 ST 2018-09-16 04:00:00  6045820   
51    

In [21]:
df_1_AV= df[df.STATION == '1 AV']
df_1_AV

C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
30340  H007  R248  00-00-00    1 AV        L      BMT  09/15/2018  00:00:00   
30341  H007  R248  00-00-00    1 AV        L      BMT  09/15/2018  04:00:00   
30342  H007  R248  00-00-00    1 AV        L      BMT  09/15/2018  08:00:00   
30343  H007  R248  00-00-00    1 AV        L      BMT  09/15/2018  12:00:00   
30344  H007  R248  00-00-00    1 AV        L      BMT  09/15/2018  16:00:00   
30345  H007  R248  00-00-00    1 AV        L      BMT  09/15/2018  20:00:00   
30346  H007  R248  00-00-00    1 AV        L      BMT  09/16/2018  00:00:00   
30347  H007  R248  00-00-00    1 AV        L      BMT  09/16/2018  04:00:00   
30348  H007  R248  00-00-00    1 AV        L      BMT  09/16/2018  08:00:00   
30349  H007  R248  00-00-00    1 AV        L      BMT  09/16/2018  12:00:00   
30350  H007  R248  00-00-00    1 AV        L      BMT  09/16/2018  16:00:00   
30351  H007  R248  00-00-00    1 AV        L      BMT  09/16/2018  20:00:00   
30352  H007  R248  00-00-00    1 AV        L      BMT  09/17/2018  00:00:00   
30353  H007  R248  00-00-00    1 AV        L      BMT  09/17/2018  04:00:00   
30354  H007  R248  00-00-00    1 AV        L      BMT  09/17/2018  08:00:00   
30355  H007  R248  00-00-00    1 AV        L      BMT  09/17/2018  12:00:00   
30356  H007  R248  00-00-00    1 AV        L      BMT  09/17/2018  16:00:00   
30357  H007  R248  00-00-00    1 AV        L      BMT  09/17/2018  20:00:00   
30358  H007  R248  00-00-00    1 AV        L      BMT  09/18/2018  00:00:00   
30359  H007  R248  00-00-00    1 AV        L      BMT  09/18/2018  04:00:00   
30360  H007  R248  00-00-00    1 AV        L      BMT  09/18/2018  08:00:00   
30361  H007  R248  00-00-00    1 AV        L      BMT  09/18/2018  12:00:00   
30362  H007  R248  00-00-00    1 AV        L      BMT  09/18/2018  16:00:00   
30363  H007  R248  00-00-00    1 AV        L      BMT  09/18/2018  20:00:00   
30364  H007  R248  00-00-00    1 AV        L      BMT  09/19/2018  00:00:00   
30365  H007  R248  00-00-00    1 AV        L      BMT  09/19/2018  04:00:00   
30366  H007  R248  00-00-00    1 AV        L      BMT  09/19/2018  08:00:00   
30367  H007  R248  00-00-00    1 AV        L      BMT  09/19/2018  12:00:00   
30368  H007  R248  00-00-00    1 AV        L      BMT  09/19/2018  16:00:00   
30369  H007  R248  00-00-00    1 AV        L      BMT  09/19/2018  20:00:00   
30370  H007  R248  00-00-00    1 AV        L      BMT  09/20/2018  00:00:00   
30371  H007  R248  00-00-00    1 AV        L      BMT  09/20/2018  04:00:00   
30372  H007  R248  00-00-00    1 AV        L      BMT  09/20/2018  08:00:00   
30373  H007  R248  00-00-00    1 AV        L      BMT  09/20/2018  12:00:00   
30374  H007  R248  00-00-00    1 AV        L      BMT  09/20/2018  16:00:00   
30375  H007  R248  00-00-00    1 AV        L      BMT  09/20/2018  20:00:00   
30376  H007  R248  00-00-00    1 AV        L      BMT  09/21/2018  00:00:00   
30377  H007  R248  00-00-00    1 AV        L      BMT  09/21/2018  04:00:00   
30378  H007  R248  00-00-00    1 AV        L      BMT  09/21/2018  08:00:00   
30379  H007  R248  00-00-00    1 AV        L      BMT  09/21/2018  12:00:00   
30380  H007  R248  00-00-00    1 AV        L      BMT  09/21/2018  16:00:00   
30381  H007  R248  00-00-00    1 AV        L      BMT  09/21/2018  20:00:00   
30382  H007  R248  00-00-01    1 AV        L      BMT  09/15/2018  00:00:00   
30383  H007  R248  00-00-01    1 AV        L      BMT  09/15/2018  04:00:00   
30384  H007  R248  00-00-01    1 AV        L      BMT  09/15/2018  08:00:00   
30385  H007  R248  00-00-01    1 AV        L      BMT  09/15/2018  12:00:00   
30386  H007  R248  00-00-01    1 AV        L      BMT  09/15/2018  16:00:00   
30387  H007  R248  00-00-01    1 AV        L      BMT  09/15/2018  20:00:00   
30388  H007  R248  00-00-01    1 AV        L      BMT  09/16/2018  00:00:00   
30389  H007  R248  00-00-01    1 AV        L      BMT  09/

In [31]:
df_1_AV_group = df_1_AV.groupby(['C/A', 'UNIT', 'SCP', 'TIMING'])
type(df_1_AV_group)
df_1_AV_group.head()
#df_1_AV_group.columns

ENTRIES  \
C/A  UNIT SCP      TIMING                          
H007 R248 00-00-00 2018-09-15 00:00:00  13935028   
                   2018-09-15 04:00:00  13935146   
                   2018-09-15 08:00:00  13935209   
                   2018-09-15 12:00:00  13935751   
                   2018-09-15 16:00:00  13936521   

                                        EXITS                                                                 
C/A  UNIT SCP      TIMING                                                                                     
H007 R248 00-00-00 2018-09-15 00:00:00                                           15643528                     
                   2018-09-15 04:00:00                                           15643854                     
                   2018-09-15 08:00:00                                           15644198                     
                   2018-09-15 12:00:00                                           15644918                     
                   2018-09-15 16:00:00                                           15645879

In [ ]:
plt.plot()

In [39]:
df_1_AV_group = df_1_AV.groupby(['C/A', 'UNIT', 'SCP', 'STATION', 'DATE']).sum()
df_1_AV_group.head()

ENTRIES  \
C/A  UNIT SCP      STATION DATE                   
H007 R248 00-00-00 1 AV    09/15/2018  83614950   
                           09/16/2018  83630466   
                           09/17/2018  83647476   
                           09/18/2018  83670947   
                           09/19/2018  83693537   

                                       EXITS                                                                 
C/A  UNIT SCP      STATION DATE                                                                              
H007 R248 00-00-00 1 AV    09/15/2018                                           93869434                     
                           09/16/2018                                           93894711                     
                           09/17/2018                                           93917820                     
                           09/18/2018                                           93942319                     
                           09/19/2018                                           93966230